In [1]:
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold, cross_validate

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
path = 'data/winemag-clean.csv'
wine_clean = pd.read_csv(path, index_col = 0)
wine_clean.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,vintage,vintage_str_data,scaled_points
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,not_specified,not_specified,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011,2011 Avidagos Red (Douro),44.317767
2,US,"Tart and snappy, the flavors of lime flesh and...",no_designation,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013,2013 Pinot Gris (Willamette Valley),41.982247
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,not_specified,Alexander Peartree,no_twitter,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,2013 Reserve Late Harvest Riesling (Lake Mich...,57.575804
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,2012 Vintner's Reserve Wild Child Block Pinot...,41.982247
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,not_specified,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,2011,2011 Ars In Vitro Tempranillo-Merlot (Navarra),50.921302


In [3]:
wine_x = wine_clean[['designation', 'region_1', 'winery', 'price']]

In [4]:
wine_x = wine_x[wine_x.groupby('designation')['designation'].transform(len) > 20]

In [5]:
wine_x = wine_x[wine_x.groupby('region_1')['region_1'].transform(len) > 20]

In [6]:
wine_x = wine_x[wine_x.groupby('winery')['winery'].transform(len) > 20]

In [7]:
# Pick the X variables that showed the best association with price:
# after model processing - should have used 'sparse = True' setting here
designation_dummy = pd.get_dummies(wine_x['designation'], 
                                  prefix = 'designation')
region_1_dummy = pd.get_dummies(wine_x['region_1'],
                               prefix = 'region_1')
winery_dummy = pd.get_dummies(wine_x['winery'],
                             prefix = 'winery')

In [8]:
X = pd.concat([designation_dummy, region_1_dummy, winery_dummy], axis=1)
y = wine_x['price']
X.head()

,designation_120,designation_1865 Single Vineyard,designation_Alpha,designation_Anna Maria,designation_Azul Portugal,designation_Bacigalupi Vineyard,designation_Barrel Fermented,designation_Barrel Select,designation_Bien Nacido Vineyard,designation_Black Label,...,winery_Whitehall Lane,winery_Willamette Valley Vineyards,winery_William Hill Estate,winery_Williams Selyem,winery_Willm,winery_Wines & Winemakers,winery_Wolfberger,winery_Woodbridge by Robert Mondavi,winery_Woodward Canyon,winery_Zantho
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5)

In [10]:
# Again see null for this set
y_null = np.zeros_like(y_test, dtype=float)
y_null.fill(y.mean())
y_null

array([26.68790241, 26.68790241, 26.68790241, ..., 26.68790241,
       26.68790241, 26.68790241])

In [11]:
print('Null case RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_null)))

Null case RMSE: 17.66594385755445


In [12]:
# Use a random forest - 500 estimators
rfreg = RandomForestRegressor(n_estimators=500,
                              oob_score=True,
                              random_state=6292,
)

In [13]:
rfreg.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=None, oob_score=True,
                      random_state=6292, verbose=0, warm_start=False)

In [14]:
pred = rfreg.predict(X_test)

In [15]:
print('Random Forest MSE:', np.sqrt(metrics.mean_squared_error(y_test, pred)))
print('Random Forest R-squared:', metrics.r2_score(y_test, pred))
print('Random Forest OOB score:', rfreg.oob_score_)

Random Forest MSE: 7.907964693281659
Random Forest R-squared: 0.7993045740889535
Random Forest OOB score: 0.8299374435410511


In [16]:
# Finally got a good score - both R-squared on OOB

In [17]:
# Get k-folds object and check it against cross-validation
# Cross validation documentation (as opposed to cross_val_score)
# from:
# https://stackoverflow.com/questions/35876508/
#  evaluate-multiple-scores-on-sklearn-cross-val-score
# and:
# http://scikit-learn.org/stable/modules/
#  generated/sklearn.model_selection.cross_validate.html
rfreg_cross = RandomForestRegressor(n_estimators=500,
                              oob_score=True,
                              random_state=6292,
)
kf = KFold(n_splits=10, shuffle=True)
scoring = {'R2': 'r2', 'nMSE': 'neg_mean_squared_error'} 
scores = cross_validate(rfreg_cross, X, y, cv=kf, scoring=scoring, return_train_score=False)

In [18]:
# Verify that the splits occured as desired
scores.keys()

dict_keys(['fit_time', 'score_time', 'test_R2', 'test_nMSE'])

In [19]:
# Verify that we have the right number of scores
print('R2 scores:', scores['test_R2'])
print('Negative MSE scores:', scores['test_nMSE'])

R2 scores: [0.84532683 0.78647395 0.83388665 0.84530771 0.83860426 0.74257888
 0.87291917 0.77787988 0.88973498 0.80646673]
Negative MSE scores: [ -62.64803899 -104.19652044  -75.09849357  -47.32848722  -75.06665422
  -70.20068812  -43.40194344  -57.69452468  -61.72663942  -55.52760094]


In [20]:
# Total output of scores:

print('Overall R-squred of Random Forrest model for predicting price (closer to 1 is better):',
     np.mean(scores['test_R2']))
print('Overall RMSE of Random Forrest model (lower is better - specifically relative to null RMSE):',
     np.sqrt(-np.mean(scores['test_nMSE'])))

Overall R-squred of Random Forrest model for predicting price (closer to 1 is better): 0.8239179032717736
Overall RMSE of Random Forrest model (lower is better - specifically relative to null RMSE): 8.08015835888659


In [21]:
# Iterate through all possible categorical variables
wine_clean.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title',
       'variety', 'winery', 'vintage', 'vintage_str_data', 'scaled_points'],
      dtype='object')

In [22]:
variables = ['country', 'designation', 'points', 'province',
              'region_1', 'region_2', 'variety', 'winery', 'vintage']

In [23]:
# Get all combinations of variables
# itertools cobinations and chain from:
# https://stackoverflow.com/questions/464864/
# how-to-get-all-possible-combinations-of-a-list-s-elements
from itertools import chain, combinations
def all_combos(variables):
    return chain(*map(lambda x: combinations(variables, x), 
                      range(3, len(variables) + 1)))

In [24]:
combos = list(all_combos(variables))
for combo in combos:
    print(combo)

('country', 'designation', 'points')
('country', 'designation', 'province')
('country', 'designation', 'region_1')
('country', 'designation', 'region_2')
('country', 'designation', 'variety')
('country', 'designation', 'winery')
('country', 'designation', 'vintage')
('country', 'points', 'province')
('country', 'points', 'region_1')
('country', 'points', 'region_2')
('country', 'points', 'variety')
('country', 'points', 'winery')
('country', 'points', 'vintage')
('country', 'province', 'region_1')
('country', 'province', 'region_2')
('country', 'province', 'variety')
('country', 'province', 'winery')
('country', 'province', 'vintage')
('country', 'region_1', 'region_2')
('country', 'region_1', 'variety')
('country', 'region_1', 'winery')
('country', 'region_1', 'vintage')
('country', 'region_2', 'variety')
('country', 'region_2', 'winery')
('country', 'region_2', 'vintage')
('country', 'variety', 'winery')
('country', 'variety', 'vintage')
('country', 'winery', 'vintage')
('designation

In [25]:
len(list(combos))

466

In [26]:
# This is still too many - just do 3 elements at a time
combos = combinations(variables, 3)
#list(combos)
# designation, region_1, winery
variations = [('designation',), ('region_1',), ('winery',), ('points',),
              ('designation', 'region_1'), ('designation', 'winery'), ('region_1', 'winery'),
              ('designation', 'region_1', 'winery', 'points'), ('country', 'designation', 'points', 'province',
              'region_1', 'region_2', 'variety', 'winery', 'vintage')]

In [27]:
# Timing function from:
# https://stackoverflow.com/questions/1557571/
# how-do-i-get-time-of-a-python-programs-execution
import time
def evaluate_models(combos):
    for combo in combos:
        print(str(combo) + ':')
        print('-------------------------')
        
        wine_x = wine_clean[list(list(combo) + ['price'])]
        for feature in combo:
            if feature != 'points':
                wine_x = wine_x[wine_x.groupby(feature)[feature].transform(len) > 20]
        if 'points' in combo:
            if len(list(combo)) == 1:
                X = wine_x[['points']]
            else:
                combo_list = list(combo)
                combo_list.remove('points')
                X = pd.concat([wine_x[['points']],
                               pd.get_dummies(wine_x[combo_list], 
                                    prefix = combo_list)], 
                                    axis=1)
        else: 
            X = pd.get_dummies(wine_x[list(combo)], prefix=list(combo))
        
        y = wine_x['price']
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5)
        
        y_null = np.zeros_like(y_test, dtype=float)
        y_null.fill(y.mean())
        print('Null case RMSE for ' + str(combo) + ':', 
              np.sqrt(metrics.mean_squared_error(y_test, y_null)))
        
        rfreg_tune = RandomForestRegressor(n_estimators=500,
                              oob_score=True,
                              random_state=6292)
        start_time = time.time()
        rfreg_tune.fit(X_train, y_train)
        fit_time = time.time() - start_time
        
        pred = rfreg_tune.predict(X_test)
        
        print('Random Forest RMSE for ' + str(combo) + ':', 
              np.sqrt(metrics.mean_squared_error(y_test, pred)))
        print('Random Forest R-squared for ' + str(combo) + ':', 
              metrics.r2_score(y_test, pred))
        print('Random Forest OOB score for ' + str(combo) + ':', 
              rfreg.oob_score_)
        print ('Random Forest Fit time for ' + str(combo) + ':',
              fit_time)
        print()
    return 

In [ ]:
# Not running this cell on all possible combinations - takes too long
# present to show work process
evaluate_models(combos)

In [ ]:
# Same as above
evaluate_models(variations)

In [28]:
final_sets = [('designation', 'region_1', 'winery', 'points'), ('country', 'designation', 'points', 'province',
              'region_1', 'region_2', 'variety', 'winery', 'vintage')]
evaluate_models(final_sets)

('designation', 'region_1', 'winery', 'points'):
-------------------------
Null case RMSE for ('designation', 'region_1', 'winery', 'points'): 17.66594385755445
Random Forest RMSE for ('designation', 'region_1', 'winery', 'points'): 8.945230923026415
Random Forest R-squared for ('designation', 'region_1', 'winery', 'points'): 0.7432022914597596
Random Forest OOB score for ('designation', 'region_1', 'winery', 'points'): 0.8299374435410511
Random Forest Fit time for ('designation', 'region_1', 'winery', 'points'): 136.117440700531

('country', 'designation', 'points', 'province', 'region_1', 'region_2', 'variety', 'winery', 'vintage'):
-------------------------
Null case RMSE for ('country', 'designation', 'points', 'province', 'region_1', 'region_2', 'variety', 'winery', 'vintage'): 18.25503478621177
Random Forest RMSE for ('country', 'designation', 'points', 'province', 'region_1', 'region_2', 'variety', 'winery', 'vintage'): 7.833739440443397
Random Forest R-squared for ('country', '

In [29]:
all_features = ['country', 'designation', 'points', 'province',
              'region_1', 'region_2', 'variety', 'winery', 'vintage']

wine_all_x = wine_clean[all_features + ['price']]
for feature in all_features:
    if feature != 'points':
        wine_all_x = wine_all_x[wine_all_x.groupby(feature)[feature].transform(len) > 20]
all_features.remove('points')
X_all = pd.concat([wine_all_x[['points']],
              pd.get_dummies(wine_all_x[all_features], 
                             prefix = all_features)], 
                             axis=1)
y_all = wine_all_x['price']

In [30]:
rfreg_all_cross = RandomForestRegressor(n_estimators=500,
                              oob_score=True,
                              random_state=6292,
)
kf = KFold(n_splits=10, shuffle=True)
scoring = {'R2': 'r2', 'nMSE': 'neg_mean_squared_error'} 
scores_all = cross_validate(rfreg_all_cross, X_all, y_all, cv=kf, scoring=scoring, return_train_score=False)

In [31]:
print('Overall R-squred of Random Forrest model for predicting price (closer to 1 is better):',
     np.mean(scores_all['test_R2']))
print('Overall RMSE of Random Forrest model (lower is better - specifically relative to null RMSE):',
     np.sqrt(-np.mean(scores_all['test_nMSE'])))

Overall R-squred of Random Forrest model for predicting price (closer to 1 is better): 0.8299898256941208
Overall RMSE of Random Forrest model (lower is better - specifically relative to null RMSE): 8.114267429253102
